In [1]:
import qiskit
import covalent as ct

In [2]:
from qiskit import QuantumCircuit, execute, transpile
import numpy as np

@ct.electron
def construct_circuit():    
    circuit = QuantumCircuit(7)
    
    unitary_operators = [[[1,0],[0,1]]]
    
    for operator in unitary_operators:
        circuit.unitary(operator, 0)
    
    circuit.measure_all()
    
    return circuit

@ct.electron
def construct_simulator(circuit):
    from qiskit.providers.ibmq import IBMQ
    IBMQ.enable_account('fd3db1e171470d152f0425e1e1883930285737aea3b3d3ada777c187e4a319b233bec7074960e559a18ed84c0909c0fabe6c4fe8147800ed2739221572933225')
 
    provider = IBMQ.get_provider(hub='ibm-q-community', group='mit-hackathon', project='main')
    provider.backends()

    backend = provider.get_backend('ibm_oslo')
    
    from qiskit.providers.aer import AerSimulator

    # Generate a simulator the mimics the real quantum system with the latest calibration
    simulator = AerSimulator.from_backend(backend)
    circuit = transpile(circuit, simulator)
    
    return circuit, simulator    

@ct.electron
def invoke_simulation(circuit, simulator):
    result = simulator.run(circuit).result()
    counts = result.get_counts(circuit)
    
    return circuit, result, counts
    
@ct.lattice
def test():
    circuit = construct_circuit()
    circuit, simulator = construct_simulator(circuit=circuit)
    
    return invoke_simulation(circuit=circuit, simulator=simulator)

In [ ]:
job_id = ct.dispatch(test)(name='Simulate')

In [5]:
ct.get_result(job_id, wait=True).result

In [6]:
import pickle
pickle.loads(b'\x80\x05\x95@\x19\x00\x00\x00\x00\x00\x00\x8c\x1dqiskit.circuit.quantumcircuit\x94\x8c\x0eQuantumCircuit\x94\x93\x94)\x81\x94}\x94(\x8c\n_base_name\x94\x8c\x07circuit\x94\x8c\x04name\x94\x8c\x10circuit-88-12007\x94\x8c\x05_data\x94]\x94(\x8c!qiskit.circuit.quantumcircuitdata\x94\x8c\x12CircuitInstruction\x94\x93\x94)\x81\x94N}\x94(\x8c\toperation\x94\x8c\x16qiskit.circuit.barrier\x94\x8c\x07Barrier\x94\x93\x94)\x81\x94}\x94(\x8c\x06_label\x94N\x8c\x05_name\x94\x8c\x07barrier\x94\x8c\x0b_num_qubits\x94K\x07\x8c\x0b_num_clbits\x94K\x00\x8c\x07_params\x94]\x94\x8c\tcondition\x94N\x8c\x0b_definition\x94N\x8c\t_duration\x94N\x8c\x05_unit\x94\x8c\x02dt\x94ub\x8c\x06qubits\x94(\x8c\x1eqiskit.circuit.quantumregister\x94\x8c\x05Qubit\x94\x93\x94)\x81\x94N}\x94(\x8c\x05_repr\x94\x8c!Qubit(QuantumRegister(7, \'q\'), 0)\x94\x8c\x05_hash\x94\x8a\x08A\xe5<I\x97G\xb8]\x8c\t_register\x94h#\x8c\x0fQuantumRegister\x94\x93\x94)\x81\x94(\x8c\x01q\x94K\x07\x8a\x08\xaf\xec;\xcf\xda%\x13\xe5\x8c\x17QuantumRegister(7, \'q\')\x94]\x94(h&h%)\x81\x94N}\x94(h(\x8c!Qubit(QuantumRegister(7, \'q\'), 1)\x94h*\x8a\x08\x9aL\x8d\xb2\xa4\xf5\xfdzh+h.\x8c\x06_index\x94K\x01u\x86\x94bh%)\x81\x94N}\x94(h(\x8c!Qubit(QuantumRegister(7, \'q\'), 2)\x94h*\x8a\x08l\xe9\xf1\x15\xbdD\x167h+h.h5K\x02u\x86\x94bh%)\x81\x94N}\x94(h(\x8c!Qubit(QuantumRegister(7, \'q\'), 3)\x94h*\x8a\x08\xc5PB\xff\x86\rf\x91h+h.h5K\x03u\x86\x94bh%)\x81\x94N}\x94(h(\x8c!Qubit(QuantumRegister(7, \'q\'), 4)\x94h*\x8a\x08\x1e\xb8\x92\xe8P\xd6\xb5\xebh+h.h5K\x04u\x86\x94bh%)\x81\x94N}\x94(h(\x8c!Qubit(QuantumRegister(7, \'q\'), 5)\x94h*\x8a\x08\xf0T\xf7\xcb\xac\n\xc4jh+h.h5K\x05u\x86\x94bh%)\x81\x94N}\x94(h(\x8c!Qubit(QuantumRegister(7, \'q\'), 6)\x94h*\x8a\x08I\xbcG\xb5v\xd3\x13\xc5h+h.h5K\x06u\x86\x94bet\x94bh5K\x00u\x86\x94bh2h7h;h?hChGt\x94\x8c\x06clbits\x94)\x8c\x14_legacy_format_cache\x94h\x14]\x94(h&h2h7h;h?hChGe]\x94\x87\x94u\x86\x94bh\r)\x81\x94N}\x94(h\x10\x8c\x16qiskit.circuit.measure\x94\x8c\x07Measure\x94\x93\x94)\x81\x94}\x94(h\x17\x8c\x07measure\x94h\x19K\x01h\x1aK\x01h\x1b]\x94h\x16Nh\x1dNh\x1eNh\x1fNh h!ubh"h&\x85\x94hN\x8c qiskit.circuit.classicalregister\x94\x8c\x05Clbit\x94\x93\x94)\x81\x94N}\x94(h(\x8c&Clbit(ClassicalRegister(7, \'meas\'), 0)\x94h*\x8a\x08\x1d4C/\xea\xaaKfh+h^\x8c\x11ClassicalRegister\x94\x93\x94)\x81\x94(\x8c\x04meas\x94K\x07\x8a\x08J\x013\xdf"m\xd1\xd5\x8c\x1cClassicalRegister(7, \'meas\')\x94]\x94(hah`)\x81\x94N}\x94(h(\x8c&Clbit(ClassicalRegister(7, \'meas\'), 1)\x94h*\x8a\x08\xef\xd0\xa7\x12F\xdfY\xe5h+hfh5K\x01u\x86\x94bh`)\x81\x94N}\x94(h(\x8c&Clbit(ClassicalRegister(7, \'meas\'), 2)\x94h*\x8a\x08H8\xf8\xfb\x0f\xa8\xa9?h+hfh5K\x02u\x86\x94bh`)\x81\x94N}\x94(h(\x8c&Clbit(ClassicalRegister(7, \'meas\'), 3)\x94h*\x8a\x08\x1a\xd5\\_(\xf7\xc1\xfbh+hfh5K\x03u\x86\x94bh`)\x81\x94N}\x94(h(\x8c&Clbit(ClassicalRegister(7, \'meas\'), 4)\x94h*\x8a\x08s<\xadH\xf2\xbf\x11Vh+hfh5K\x04u\x86\x94bh`)\x81\x94N}\x94(h(\x8c&Clbit(ClassicalRegister(7, \'meas\'), 5)\x94h*\x8a\x08E\xd9\x11\xac\n\x0f*\x12h+hfh5K\x05u\x86\x94bh`)\x81\x94N}\x94(h(\x8c&Clbit(ClassicalRegister(7, \'meas\'), 6)\x94h*\x8a\x08\x9e@b\x15\x18\xbdo/h+hfh5K\x06u\x86\x94bet\x94bh5K\x00u\x86\x94b\x85\x94hOhY]\x94h&a]\x94haa\x87\x94u\x86\x94bh\r)\x81\x94N}\x94(h\x10hX)\x81\x94}\x94(h\x17h[h\x19K\x01h\x1aK\x01h\x1b]\x94h\x16Nh\x1dNh\x1eNh\x1fNh h!ubh"h2\x85\x94hNhj\x85\x94hOh\x8b]\x94h2a]\x94hja\x87\x94u\x86\x94bh\r)\x81\x94N}\x94(h\x10hX)\x81\x94}\x94(h\x17h[h\x19K\x01h\x1aK\x01h\x1b]\x94h\x16Nh\x1dNh\x1eNh\x1fNh h!ubh"h7\x85\x94hNhn\x85\x94hOh\x96]\x94h7a]\x94hna\x87\x94u\x86\x94bh\r)\x81\x94N}\x94(h\x10hX)\x81\x94}\x94(h\x17h[h\x19K\x01h\x1aK\x01h\x1b]\x94h\x16Nh\x1dNh\x1eNh\x1fNh h!ubh"h;\x85\x94hNhr\x85\x94hOh\xa1]\x94h;a]\x94hra\x87\x94u\x86\x94bh\r)\x81\x94N}\x94(h\x10hX)\x81\x94}\x94(h\x17h[h\x19K\x01h\x1aK\x01h\x1b]\x94h\x16Nh\x1dNh\x1eNh\x1fNh h!ubh"h?\x85\x94hNhv\x85\x94hOh\xac]\x94h?a]\x94hva\x87\x94u\x86\x94bh\r)\x81\x94N}\x94(h\x10hX)\x81\x94}\x94(h\x17h[h\x19K\x01h\x1aK\x01h\x1b]\x94h\x16Nh\x1dNh\x1eNh\x1fNh h!ubh"hC\x85\x94hNhz\x85\x94hOh\xb7]\x94hCa]\x94hza\x87\x94u\x86\x94bh\r)\x81\x94N}\x94(h\x10hX)\x81\x94}\x94(h\x17h[h\x19K\x01h\x1aK\x01h\x1b]\x94h\x16Nh\x1dNh\x1eNh\x1fNh h!ubh"hG\x85\x94hNh~\x85\x94hOh\xc2]\x94hGa]\x94h~a\x87\x94u\x86\x94be\x8c\x0f_op_start_times\x94N\x8c\x14_control_flow_scopes\x94]\x94\x8c\x05qregs\x94]\x94h.a\x8c\x05cregs\x94]\x94hfa\x8c\x07_qubits\x94]\x94(h&h2h7h;h?hChGe\x8c\x07_clbits\x94]\x94(hahjhnhrhvhzh~e\x8c\x0e_qubit_indices\x94}\x94(h&h\x00\x8c\x0cBitLocations\x94\x93\x94K\x00]\x94h.K\x00\x86\x94a\x86\x94\x81\x94h2h\xd9K\x01]\x94h.K\x01\x86\x94a\x86\x94\x81\x94h7h\xd9K\x02]\x94h.K\x02\x86\x94a\x86\x94\x81\x94h;h\xd9K\x03]\x94h.K\x03\x86\x94a\x86\x94\x81\x94h?h\xd9K\x04]\x94h.K\x04\x86\x94a\x86\x94\x81\x94hCh\xd9K\x05]\x94h.K\x05\x86\x94a\x86\x94\x81\x94hGh\xd9K\x06]\x94h.K\x06\x86\x94a\x86\x94\x81\x94u\x8c\x0e_clbit_indices\x94}\x94(hah\xd9K\x00]\x94hfK\x00\x86\x94a\x86\x94\x81\x94hjh\xd9K\x01]\x94hfK\x01\x86\x94a\x86\x94\x81\x94hnh\xd9K\x02]\x94hfK\x02\x86\x94a\x86\x94\x81\x94hrh\xd9K\x03]\x94hfK\x03\x86\x94a\x86\x94\x81\x94hvh\xd9K\x04]\x94hfK\x04\x86\x94a\x86\x94\x81\x94hzh\xd9K\x05]\x94hfK\x05\x86\x94a\x86\x94\x81\x94h~h\xd9K\x06]\x94hfK\x06\x86\x94a\x86\x94\x81\x94u\x8c\t_ancillas\x94]\x94\x8c\r_calibrations\x94\x8c\x0bcollections\x94\x8c\x0bdefaultdict\x94\x93\x94\x8c\x08builtins\x94\x8c\x04dict\x94\x93\x94\x85\x94R\x94\x8c\x10_parameter_table\x94\x8c\x1dqiskit.circuit.parametertable\x94\x8c\x0eParameterTable\x94\x93\x94)\x81\x94N}\x94(\x8c\x06_table\x94}\x94\x8c\x05_keys\x94\x8f\x94\x8c\x06_names\x94\x8f\x94u\x86\x94b\x8c\x0b_parameters\x94]\x94\x8c\x07_layout\x94\x8c\x18qiskit.transpiler.layout\x94\x8c\x0fTranspileLayout\x94\x93\x94)\x81\x94}\x94(\x8c\x0einitial_layout\x94j/\x01\x00\x00\x8c\x06Layout\x94\x93\x94)\x81\x94N}\x94(\x8c\x05_regs\x94]\x94(h-)\x81\x94(h/K\x07\x8a\x08\x8b\xc7\xd6\xae\xe6/\xdew\x8c\x17QuantumRegister(7, \'q\')\x94]\x94(h%)\x81\x94N}\x94(h(\x8c!Qubit(QuantumRegister(7, \'q\'), 0)\x94h*\x8a\x08\x0b*\xa2\x1e\x0ex\xd1\x06h+j;\x01\x00\x00h5K\x00u\x86\x94bh%)\x81\x94N}\x94(h(\x8c!Qubit(QuantumRegister(7, \'q\'), 1)\x94h*\x8a\x08d\x91\xf2\x07\xd8@!ah+j;\x01\x00\x00h5K\x01u\x86\x94bh%)\x81\x94N}\x94(h(\x8c!Qubit(QuantumRegister(7, \'q\'), 2)\x94h*\x8a\x08\xbd\xf8B\xf1\xa1\tq\xbbh+j;\x01\x00\x00h5K\x02u\x86\x94bh%)\x81\x94N}\x94(h(\x8c!Qubit(QuantumRegister(7, \'q\'), 3)\x94h*\x8a\x08\x8f\x95\xa7T\xbaX\x89wh+j;\x01\x00\x00h5K\x03u\x86\x94bh%)\x81\x94N}\x94(h(\x8c!Qubit(QuantumRegister(7, \'q\'), 4)\x94h*\x8a\x08\xe8\xfc\xf7\xbd\xc7\x06\xcf\x94h+j;\x01\x00\x00h5K\x04u\x86\x94bh%)\x81\x94N}\x94(h(\x8c!Qubit(QuantumRegister(7, \'q\'), 5)\x94h*\x8a\x08\xba\x99\\!\xe0U\xe7Ph+j;\x01\x00\x00h5K\x05u\x86\x94bh%)\x81\x94N}\x94(h(\x8c!Qubit(QuantumRegister(7, \'q\'), 6)\x94h*\x8a\x08\x13\x01\xad\n\xaa\x1e7\xabh+j;\x01\x00\x00h5K\x06u\x86\x94bet\x94bj;\x01\x00\x00e\x8c\x04_p2v\x94}\x94(K\x00j>\x01\x00\x00K\x01jB\x01\x00\x00K\x02jF\x01\x00\x00K\x03jJ\x01\x00\x00K\x04jN\x01\x00\x00K\x05jR\x01\x00\x00K\x06jV\x01\x00\x00u\x8c\x04_v2p\x94}\x94(j>\x01\x00\x00K\x00jB\x01\x00\x00K\x01jF\x01\x00\x00K\x02jJ\x01\x00\x00K\x03jN\x01\x00\x00K\x04jR\x01\x00\x00K\x05jV\x01\x00\x00K\x06uu\x86\x94b\x8c\x13input_qubit_mapping\x94}\x94(j>\x01\x00\x00K\x00jB\x01\x00\x00K\x01jF\x01\x00\x00K\x02jJ\x01\x00\x00K\x03jN\x01\x00\x00K\x04jR\x01\x00\x00K\x05jV\x01\x00\x00K\x06u\x8c\x0cfinal_layout\x94Nub\x8c\r_global_phase\x94K\x00\x8c\x08duration\x94N\x8c\x04unit\x94\x8c\x02dt\x94\x8c\t_metadata\x94N\x8c\x14_clbit_write_latency\x94N\x8c\x14_conditional_latency\x94Nub\x8c\x14qiskit.result.result\x94\x8c\x06Result\x94\x93\x94)\x81\x94}\x94(jg\x01\x00\x00}\x94(\x8c\x08metadata\x94}\x94(\x8c\ntime_taken\x94G?\xd8\x0bww\xb0\xbb\x9f\x8c\x12time_taken_execute\x94G?\xb7\xda\xf8<D\xdd]\x8c\x08mpi_rank\x94K\x00\x8c\x11num_mpi_processes\x94K\x01\x8c\x11max_gpu_memory_mb\x94K\x00\x8c\rmax_memory_mb\x94M\x07\x0e\x8c\x14parallel_experiments\x94K\x01\x8c\x14time_taken_load_qobj\x94G?\xd2\x14m\xd7\x04Do\x8c\x1dnum_processes_per_experiments\x94K\x01\x8c\x0bomp_enabled\x94\x88u\x8c\ntime_taken\x94G?\xd8B\'\x00\x00\x00\x00u\x8c\x0cbackend_name\x94\x8c\x17aer_simulator(ibm_oslo)\x94\x8c\x0fbackend_version\x94\x8c\x061.0.17\x94\x8c\x07qobj_id\x94\x8c$9f15b0e2-3fe6-4a63-88fa-804336be5439\x94\x8c\x06job_id\x94\x8c$d542b52c-cf5a-41ef-a89a-1586011bd34f\x94\x8c\x07success\x94\x88\x8c\x07results\x94]\x94\x8c\x14qiskit.result.models\x94\x8c\x10ExperimentResult\x94\x93\x94)\x81\x94}\x94(jg\x01\x00\x00}\x94(\x8c\x0eseed_simulator\x94J\x9cJN\x02\x8c\x08metadata\x94}\x94(\x8c\x05noise\x94\x8c\x07superop\x94\x8c\x1abatched_shots_optimization\x94\x89\x8c\x10measure_sampling\x94\x88\x8c\x0eparallel_shots\x94K\x01\x8c\x0fremapped_qubits\x94\x89\x8c\x13active_input_qubits\x94]\x94(K\x00K\x01K\x02K\x03K\x04K\x05K\x06e\x8c\nnum_clbits\x94K\x07\x8c\x15parallel_state_update\x94K\x01\x8c\x13sample_measure_time\x94G?/OP\xa0+\x84\x12\x8c\nnum_qubits\x94K\x07\x8c\x06device\x94\x8c\x03CPU\x94\x8c\x0finput_qubit_map\x94]\x94(]\x94(K\x06K\x06e]\x94(K\x05K\x05e]\x94(K\x04K\x04e]\x94(K\x03K\x03e]\x94(K\x02K\x02e]\x94(K\x01K\x01e]\x94(K\x00K\x00ee\x8c\x06method\x94\x8c\x0edensity_matrix\x94\x8c\x06fusion\x94}\x94(\x8c\x07applied\x94\x89\x8c\x10max_fused_qubits\x94K\x02\x8c\tthreshold\x94K\x07\x8c\x07enabled\x94\x88uu\x8c\ntime_taken\x94G?\xb3\xc1\xcecr7\xa2u\x8c\x05shots\x94M\x00\x04j\x85\x01\x00\x00\x88\x8c\x04data\x94j\x88\x01\x00\x00\x8c\x14ExperimentResultData\x94\x93\x94)\x81\x94}\x94(\x8c\x10_data_attributes\x94]\x94\x8c\x06counts\x94aj\xb9\x01\x00\x00}\x94(\x8c\x040x24\x94K\x01\x8c\x030xa\x94K\x01\x8c\x040x4a\x94K\x01\x8c\x030x2\x94K\t\x8c\x030x0\x94M\t\x03\x8c\x040x40\x94Kw\x8c\x040x20\x94K\n\x8c\x040x10\x94K5\x8c\x030x4\x94K\t\x8c\x030x1\x94K\x04\x8c\x040x48\x94K\x04\x8c\x040x50\x94K\x0b\x8c\x030x8\x94K\x13\x8c\x030x6\x94K\x02\x8c\x040x44\x94K\x03\x8c\x040x60\x94K\x01uub\x8c\nmeas_level\x94\x8c\x11qiskit.qobj.utils\x94\x8c\tMeasLevel\x94\x93\x94K\x02\x85\x94R\x94\x8c\x06header\x94\x8c\x12qiskit.qobj.common\x94\x8c\x14QobjExperimentHeader\x94\x93\x94)R\x94}\x94(\x8c\x0cclbit_labels\x94]\x94(]\x94(\x8c\x04meas\x94K\x00e]\x94(\x8c\x04meas\x94K\x01e]\x94(\x8c\x04meas\x94K\x02e]\x94(\x8c\x04meas\x94K\x03e]\x94(\x8c\x04meas\x94K\x04e]\x94(\x8c\x04meas\x94K\x05e]\x94(\x8c\x04meas\x94K\x06ee\x8c\ncreg_sizes\x94]\x94]\x94(\x8c\x04meas\x94K\x07ea\x8c\x0cglobal_phase\x94G\x00\x00\x00\x00\x00\x00\x00\x00\x8c\x0cmemory_slots\x94K\x07\x8c\x08metadata\x94}\x94\x8c\x08n_qubits\x94K\x07\x8c\x04name\x94\x8c\x10circuit-88-12007\x94\x8c\nqreg_sizes\x94]\x94]\x94(h/K\x07ea\x8c\x0cqubit_labels\x94]\x94(]\x94(h/K\x00e]\x94(h/K\x01e]\x94(h/K\x02e]\x94(h/K\x03e]\x94(h/K\x04e]\x94(h/K\x05e]\x94(h/K\x06eeub\x8c\x06status\x94\x8c\x04DONE\x94uba\x8c\x04date\x94\x8c\x1a2023-01-29T01:48:45.408863\x94j\xfe\x01\x00\x00\x8c\tCOMPLETED\x94j\xd1\x01\x00\x00j\xd2\x01\x00\x00\x8c\nQobjHeader\x94\x93\x94)R\x94}\x94(\x8c\x0cbackend_name\x94\x8c\x17aer_simulator(ibm_oslo)\x94\x8c\x0fbackend_version\x94\x8c\x061.0.17\x94ubub\x8c\x14qiskit.result.counts\x94\x8c\x06Counts\x94\x93\x94)\x81\x94(\x8c\x070100100\x94K\x01\x8c\x070001010\x94K\x01\x8c\x071001010\x94K\x01\x8c\x070000010\x94K\t\x8c\x070000000\x94M\t\x03\x8c\x071000000\x94Kw\x8c\x070100000\x94K\n\x8c\x070010000\x94K5\x8c\x070000100\x94K\t\x8c\x070000001\x94K\x04\x8c\x071001000\x94K\x04\x8c\x071010000\x94K\x0b\x8c\x070001000\x94K\x13\x8c\x070000110\x94K\x02\x8c\x071000100\x94K\x03\x8c\x071100000\x94K\x01u}\x94(\x8c\x07hex_raw\x94}\x94(j\xbb\x01\x00\x00K\x01j\xbc\x01\x00\x00K\x01j\xbd\x01\x00\x00K\x01j\xbe\x01\x00\x00K\tj\xbf\x01\x00\x00M\t\x03j\xc0\x01\x00\x00Kwj\xc1\x01\x00\x00K\nj\xc2\x01\x00\x00K5j\xc3\x01\x00\x00K\tj\xc4\x01\x00\x00K\x04j\xc5\x01\x00\x00K\x04j\xc6\x01\x00\x00K\x0bj\xc7\x01\x00\x00K\x13j\xc8\x01\x00\x00K\x02j\xc9\x01\x00\x00K\x03j\xca\x01\x00\x00K\x01u\x8c\x07int_raw\x94}\x94(K$K\x01K\nK\x01KJK\x01K\x02K\tK\x00M\t\x03K@KwK K\nK\x10K5K\x04K\tK\x01K\x04KHK\x04KPK\x0bK\x08K\x13K\x06K\x02KDK\x03K`K\x01u\x8c\ncreg_sizes\x94j\xe8\x01\x00\x00\x8c\x0cmemory_slots\x94K\x07j|\x01\x00\x00Nub\x87\x94.')

(<qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7feb486f0940>,
 Result(backend_name='aer_simulator(ibm_oslo)', backend_version='1.0.17', qobj_id='9f15b0e2-3fe6-4a63-88fa-804336be5439', job_id='d542b52c-cf5a-41ef-a89a-1586011bd34f', success=True, results=[ExperimentResult(shots=1024, success=True, meas_level=2, data=ExperimentResultData(counts={'0x24': 1, '0xa': 1, '0x4a': 1, '0x2': 9, '0x0': 777, '0x40': 119, '0x20': 10, '0x10': 53, '0x4': 9, '0x1': 4, '0x48': 4, '0x50': 11, '0x8': 19, '0x6': 2, '0x44': 3, '0x60': 1}), header=QobjExperimentHeader(clbit_labels=[['meas', 0], ['meas', 1], ['meas', 2], ['meas', 3], ['meas', 4], ['meas', 5], ['meas', 6]], creg_sizes=[['meas', 7]], global_phase=0.0, memory_slots=7, metadata={}, n_qubits=7, name='circuit-88-12007', qreg_sizes=[['q', 7]], qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4], ['q', 5], ['q', 6]]), status=DONE, seed_simulator=38685340, metadata={'noise': 'superop', 'batched_shots_optimization': False, 'measure_samp